## Chaque ligne du csv est une arête du graphe : Graphe card <--> marchand

In [1]:
import pandas as pd 
import numpy as np
import networkx as nx 
import torch
from torch_geometric.data import Data
from torch_geometric.nn import VGAE, GCNConv
from torch_geometric.nn import InnerProductDecoder
import scipy.sparse as sp
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from datetime import datetime
import torch.nn.functional as F
from torch import nn

/home/ensai/.local/lib/python3.10/site-packages/torch/__config__.py:9: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._show_config()
/home/ensai/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
train_fraud = pd.read_csv("../data/fraudTrain.csv")

In [3]:
# ==================== FEATURE ENGINEERING ====================

# Identifier la colonne de date (peut avoir différents noms)
date_col = None
for col in train_fraud.columns:
    if 'trans' in col.lower() and ('date' in col.lower() or 'time' in col.lower()):
        date_col = col
        break

if date_col is None:
    print("Colonne de date non trouvée. Colonnes disponibles:")
    print(train_fraud.columns.tolist())
    date_col = 'trans_date_trans_time'  # Valeur par défaut

print(f" Colonne de date utilisée: {date_col}")

# Convertir en datetime
train_fraud['trans_datetime'] = pd.to_datetime(train_fraud[date_col])
train_fraud['hour'] = train_fraud['trans_datetime'].dt.hour
train_fraud['day_of_week'] = train_fraud['trans_datetime'].dt.dayofweek
train_fraud['day_of_month'] = train_fraud['trans_datetime'].dt.day
train_fraud['is_weekend'] = train_fraud['day_of_week'].isin([5, 6]).astype(int)

# Encoder la catégorie
le_category = LabelEncoder()
train_fraud['category_encoded'] = le_category.fit_transform(train_fraud['category'])

print(" Catégories uniques:")
print(f"Nombre de catégories: {train_fraud['category'].nunique()}")
print("\nTop 10 catégories:")
print(train_fraud['category'].value_counts().head(10))

# Analyser les distances géographiques
print(" Variables géographiques disponibles:")
geo_cols = ['city', 'state', 'street', 'lat', 'long']
for col in geo_cols:
    if col in train_fraud.columns:
        print(f"  - {col}: {train_fraud[col].nunique()} valeurs uniques")

 Colonne de date utilisée: trans_date_trans_time
 Catégories uniques:
Nombre de catégories: 14

Top 10 catégories:
category
gas_transport     131659
grocery_pos       123638
home              123115
shopping_pos      116672
kids_pets         113035
shopping_net       97543
entertainment      94014
food_dining        91461
personal_care      90758
health_fitness     85879
Name: count, dtype: int64
 Variables géographiques disponibles:
  - city: 894 valeurs uniques
  - state: 51 valeurs uniques
  - street: 983 valeurs uniques
  - lat: 968 valeurs uniques
  - long: 969 valeurs uniques


In [4]:
# création des noeuds 
# Cartes
cards = train_fraud['cc_num'].astype(str).unique()
card2id = {c: i for i, c in enumerate(cards)}

# Marchands
merchants = train_fraud['merchant'].unique()
merchant2id = {m: i + len(card2id) for i, m in enumerate(merchants)}
print(merchant2id)
num_nodes = len(card2id) + len(merchant2id)
print("Nombre total de nœuds :", num_nodes)


{'fraud_Rippin, Kub and Mann': 983, 'fraud_Heller, Gutmann and Zieme': 984, 'fraud_Lind-Buckridge': 985, 'fraud_Kutch, Hermiston and Farrell': 986, 'fraud_Keeling-Crist': 987, 'fraud_Stroman, Hudson and Erdman': 988, 'fraud_Rowe-Vandervort': 989, 'fraud_Corwin-Collins': 990, 'fraud_Herzog Ltd': 991, 'fraud_Schoen, Kuphal and Nitzsche': 992, 'fraud_Rutherford-Mertz': 993, 'fraud_Kerluke-Abshire': 994, 'fraud_Lockman Ltd': 995, 'fraud_Kiehn Inc': 996, 'fraud_Beier-Hyatt': 997, 'fraud_Schmidt and Sons': 998, 'fraud_Lebsack and Sons': 999, 'fraud_Mayert Group': 1000, 'fraud_Konopelski, Schneider and Hartmann': 1001, 'fraud_Schultz, Simonis and Little': 1002, 'fraud_Bauch-Raynor': 1003, 'fraud_Harris Inc': 1004, 'fraud_Kling-Grant': 1005, 'fraud_Pacocha-Bauch': 1006, 'fraud_Lesch Ltd': 1007, 'fraud_Kunde-Sanford': 1008, "fraud_Deckow-O'Conner": 1009, 'fraud_Bruen-Yost': 1010, 'fraud_Kunze Inc': 1011, 'fraud_Nitzsche, Kessler and Wolff': 1012, 'fraud_Kihn, Abernathy and Douglas': 1013, 'frau

In [5]:
#créer les arêtes (card <--> marchand)

edge_data = train_fraud.groupby(['cc_num', 'merchant']).agg({
    'amt': ['sum', 'mean', 'count'],
    'is_fraud': 'max'  # is_fraud est utilisé pour labels seulement
}).reset_index()

edges = []
edge_labels = []

for _, row in edge_data.iterrows():
    card_id = card2id[str(row[('cc_num', '')])]
    merchant_id = merchant2id[row[('merchant', '')]]
    edges.append([card_id, merchant_id])
    edge_labels.append(row[('is_fraud', 'max')])

edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

print(f" Arêtes:")
print(f"  Avant agrégation: {len(train_fraud):,} transactions")
print(f"  Après agrégation: {len(edges):,} paires uniques")
print(f"  Réduction: {(1 - len(edges)/len(train_fraud))*100:.1f}%")

# Bidirectionnel
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
edge_labels = torch.tensor(edge_labels + edge_labels, dtype=torch.float)

print(f"  Après bidirection: {edge_index.shape[1]:,} arêtes")


 Arêtes:
  Avant agrégation: 1,296,675 transactions
  Après agrégation: 479,072 paires uniques
  Réduction: 63.1%
  Après bidirection: 958,144 arêtes


In [6]:
# construire les features des noeuds 

# features cartes

card_features = train_fraud.groupby('cc_num').agg({
    # Montants
    'amt': ['count', 'sum', 'mean', 'std', 'min', 'max'],
    
    # Temporelles
    'hour': ['mean', 'std'],
    'is_weekend': 'mean',
    'day_of_week': lambda x: x.mode()[0] if len(x.mode()) > 0 else 0,
    
    # Catégories (diversité)
    'category': 'nunique',
    'merchant': 'nunique',
    
    # Géographiques
    'state': 'nunique',
    'city': 'nunique'
}).fillna(0)

# Aplatir les colonnes multi-niveaux
card_features.columns = ['_'.join(col).strip() for col in card_features.columns]

print(" Features des cartes:")
print(card_features.columns.tolist())
print(f"Dimension: {card_features.shape}")
print(" Exemple:")
print(card_features.head())

# features marchands 
merchant_features = train_fraud.groupby('merchant').agg({
    # Montants
    'amt': ['count', 'sum', 'mean', 'std', 'min', 'max'],
    
    # Temporelles
    'hour': ['mean', 'std'],
    'is_weekend': 'mean',
    
    # Diversité clients
    'cc_num': 'nunique',
    
    # Catégorie dominante
    'category': lambda x: le_category.transform([x.mode()[0]])[0] if len(x.mode()) > 0 else 0,
    
    # Géographique
    #'state': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown',
    'city': 'nunique'
}).fillna(0)



merchant_features.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col 
                             for col in merchant_features.columns]

print(" Features des marchands:")
print(merchant_features.columns.tolist())
print(f"Dimension: {merchant_features.shape}")
print(" Exemple:")
print(merchant_features.head())


 Features des cartes:
['amt_count', 'amt_sum', 'amt_mean', 'amt_std', 'amt_min', 'amt_max', 'hour_mean', 'hour_std', 'is_weekend_mean', 'day_of_week_<lambda>', 'category_nunique', 'merchant_nunique', 'state_nunique', 'city_nunique']
Dimension: (983, 14)
 Exemple:
             amt_count    amt_sum    amt_mean      amt_std  amt_min   amt_max  \
cc_num                                                                          
60416207185       1518   85043.47   56.023366   122.632635     1.02   3075.09   
60422928733       1531  105640.20   69.000784   102.681962     1.04   1290.37   
60423098130        510   58673.63  115.046333  1202.988005     1.01  27119.77   
60427851591        528   59129.61  111.987898   143.310653     8.86   1164.36   
60487002085        496   25160.11   50.726028    65.843969     1.02    750.39   

             hour_mean  hour_std  is_weekend_mean  day_of_week_<lambda>  \
cc_num                                                                    
60416207185  13.68

In [7]:
    ###################################################################################################
    ### Feature Supplémentaire avancée 


    # Pour les cartes : Variabilité temporelle
card_time_features = train_fraud.groupby('cc_num').apply(
        lambda x: pd.Series({
            'hour_range': x['hour'].max() - x['hour'].min(),
            'active_days': x['trans_datetime'].dt.date.nunique(),
            'avg_daily_transactions': len(x) / max(1, x['trans_datetime'].dt.date.nunique()),
            'night_transactions_pct': (x['hour'].between(0, 6).sum() / len(x)) * 100 if len(x) > 0 else 0
        })
    ).fillna(0)

print(" Features temporelles avancées pour cartes:")
print(card_time_features.head())

# Pour les marchands : Patterns de montants
merchant_amt_features = train_fraud.groupby('merchant').apply(
    lambda x: pd.Series({
        'amt_cv': (x['amt'].std() / x['amt'].mean()) if x['amt'].mean() > 0 else 0,  # Coef variation
        'amt_range': x['amt'].max() - x['amt'].min(),
        'high_amt_pct': (x['amt'] > x['amt'].quantile(0.75)).sum() / len(x) * 100
    })
).fillna(0)

print(" Features montants avancées pour marchands:")
print(merchant_amt_features.head())

/tmp/ipykernel_1762114/1090195648.py:6: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  card_time_features = train_fraud.groupby('cc_num').apply(


 Features temporelles avancées pour cartes:
             hour_range  active_days  avg_daily_transactions  \
cc_num                                                         
60416207185        23.0        498.0                3.048193   
60422928733        23.0        484.0                3.163223   
60423098130        23.0        318.0                1.603774   
60427851591        23.0        318.0                1.660377   
60487002085        23.0        319.0                1.554859   

             night_transactions_pct  
cc_num                               
60416207185               17.720685  
60422928733               35.009798  
60423098130               27.254902  
60427851591               25.189394  
60487002085               19.758065  
 Features montants avancées pour marchands:
                                    amt_cv  amt_range  high_amt_pct
merchant                                                           
fraud_Abbott-Rogahn               1.074634     654.58     25.

/tmp/ipykernel_1762114/1090195648.py:19: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  merchant_amt_features = train_fraud.groupby('merchant').apply(


In [8]:
###############################################################################################################################
# fusion des features + normalisation

# Combiner toutes les features pour cartes
card_features_final = pd.concat([card_features, card_time_features], axis=1).fillna(0)

# Combiner pour marchands
merchant_features_final = pd.concat([merchant_features, merchant_amt_features], axis=1).fillna(0)

print(f" Features finales cartes: {card_features_final.shape[1]} colonnes")
print(f" Features finales marchands: {merchant_features_final.shape[1]} colonnes")

# Aligner les dimensions (padding ou sélection)
n_card_features = card_features_final.shape[1]
n_merchant_features = merchant_features_final.shape[1]

print(f" Alignement des dimensions:")
print(f"  Cartes: {n_card_features} features")
print(f"  Marchands: {n_merchant_features} features")

if n_card_features != n_merchant_features:
    print("  Les dimensions diffèrent. Options:")
    print("  1. Padding avec des zéros (simple mais sous-optimal)")
    print("  2. Sélectionner features communes (perte d'info)")
    print("  3. Utiliser un graphe hétérogène (recommandé)")
    
    # Option 1: Padding
    max_features = max(n_card_features, n_merchant_features)
    
    if n_card_features < max_features:
        padding = pd.DataFrame(
            0, 
            index=card_features_final.index,
            columns=[f'pad_{i}' for i in range(max_features - n_card_features)]
        )
        card_features_final = pd.concat([card_features_final, padding], axis=1)
    
    if n_merchant_features < max_features:
        padding = pd.DataFrame(
            0,
            index=merchant_features_final.index,
            columns=[f'pad_{i}' for i in range(max_features - n_merchant_features)]
        )
        merchant_features_final = pd.concat([merchant_features_final, padding], axis=1)

# Convertir en arrays
X_cards = card_features_final.values
X_merchants = merchant_features_final.values

# NORMALISATION 
X = np.vstack([X_cards, X_merchants])

scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

print(f" Statistiques après normalisation:")
print(f"  Mean: {X_normalized.mean():.6f}")
print(f"  Std: {X_normalized.std():.6f}")
print(f"  Min: {X_normalized.min():.6f}")
print(f"  Max: {X_normalized.max():.6f}")

X_tensor = torch.tensor(X_normalized, dtype=torch.float)

 Features finales cartes: 18 colonnes
 Features finales marchands: 15 colonnes
 Alignement des dimensions:
  Cartes: 18 features
  Marchands: 15 features
  Les dimensions diffèrent. Options:
  1. Padding avec des zéros (simple mais sous-optimal)
  2. Sélectionner features communes (perte d'info)
  3. Utiliser un graphe hétérogène (recommandé)
 Statistiques après normalisation:
  Mean: -0.000000
  Std: 1.000000
  Min: -16.211226
  Max: 12.674216


In [9]:
# création de l'objet graphe qui est l'entrée dans le vgae
data_train = Data(
    x=X_tensor,
    edge_index=edge_index,
    y=edge_labels,  # Labels pour validation
    num_nodes=num_nodes
)

print("\n" + "="*80)
print("GRAPHE FINAL")
print("="*80)
print(data_train)

print(f"""
RÉSUMÉ:
  • Nœuds: {num_nodes:,}
  • Features par nœud: {X_tensor.shape[1]}
  • Arêtes: {edge_index.shape[1]:,}
  • Labels disponibles: {len(edge_labels):,}
  • Pas de data leakage: ✓
  • Features normalisées: ✓
  • Multi-edges agrégées: ✓
""")


GRAPHE FINAL
Data(x=[1676, 18], edge_index=[2, 958144], y=[958144], num_nodes=1676)

RÉSUMÉ:
  • Nœuds: 1,676
  • Features par nœud: 18
  • Arêtes: 958,144
  • Labels disponibles: 958,144
  • Pas de data leakage: ✓
  • Features normalisées: ✓
  • Multi-edges agrégées: ✓



## GNN supervisé appliqué au graphe hétérogène card <--> marchand

In [10]:
# classification d'arêtes 

class EdgeReadout(nn.Module):
    def __init__(self, in_dim):
        super().__init__()
        self.mlp = nn.Sequential(
                nn.Linear(in_dim*2, 32),
                nn.ReLU(),
                nn.Linear(32, 2)
        )

    def forward(self, x, edge_index):
        src, dst = edge_index
        edge_emb = torch.cat([x[src], x[dst]], dim=1)
        return self.mlp(edge_emb)

In [11]:
class SimpleGCN(nn.Module):
    def __init__(self, in_features=1, hidden=32, out_features=16):
        super().__init__()
        self.conv1 = GCNConv(in_features, hidden)
        self.conv2 = GCNConv(hidden, out_features)
        self.readout = EdgeReadout(out_features)

    def forward(self, data_train):
        x, edge_index = data_train.x, data_train.edge_index

        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.3, training=self.training)
        x = self.conv2(x, edge_index)

        logits = self.readout(x, edge_index)
        return logits

In [12]:
# Entraînement supervisé 

device = torch.device("cpu")

model = SimpleGCN(
    in_features=data_train.x.shape[1],
    hidden=32,
    out_features=16
).to(device)

data_train = data_train.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-4)
criterion = nn.CrossEntropyLoss()


In [13]:
# Boucles d'entraînement 

epochs = 100

for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    logits = model(data_train)              # (num_edges, 2)
    data_train.y = data_train.y.long()
    loss = criterion(logits, data_train.y)  # supervision fraude

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        preds = logits.argmax(dim=1)
        acc = (preds == data_train.y).float().mean().item()
        print(f"Epoch {epoch:03d} | Loss: {loss.item():.4f} | Acc: {acc:.4f}")


Epoch 000 | Loss: 0.6292 | Acc: 0.9846
Epoch 010 | Loss: 0.1484 | Acc: 0.9846
Epoch 020 | Loss: 0.0887 | Acc: 0.9846
Epoch 030 | Loss: 0.0818 | Acc: 0.9846
Epoch 040 | Loss: 0.0786 | Acc: 0.9846
Epoch 050 | Loss: 0.0776 | Acc: 0.9846
Epoch 060 | Loss: 0.0770 | Acc: 0.9846
Epoch 070 | Loss: 0.0765 | Acc: 0.9846
Epoch 080 | Loss: 0.0760 | Acc: 0.9846
Epoch 090 | Loss: 0.0756 | Acc: 0.9846


In [ ]:
# # AUC fraude sur le train 

# model.eval()
# with torch.no_grad():
#     logits = model(data_train)
#     probs = F.softmax(logits, dim=1)[:, 1]  # proba fraude

# auc = roc_auc_score(
#     data_train.y.cpu().numpy(),
#     probs.cpu().numpy()
# )

# print("AUC fraude :", auc)


AUC fraude : 0.6015495051638217


## graphe sur le csv de test et evaluation 

In [ ]:
test_fraud = pd.read_csv("../data/fraudTest.csv")

In [22]:
# ==================== FEATURE ENGINEERING ====================

# Identifier la colonne de date (peut avoir différents noms)
date_col = None
for col in test_fraud.columns:
    if 'trans' in col.lower() and ('date' in col.lower() or 'time' in col.lower()):
        date_col = col
        break

if date_col is None:
    print("Colonne de date non trouvée. Colonnes disponibles:")
    print(test_fraud.columns.tolist())
    date_col = 'trans_date_trans_time'  # Valeur par défaut

print(f" Colonne de date utilisée: {date_col}")

# Convertir en datetime
test_fraud['trans_datetime'] = pd.to_datetime(test_fraud[date_col])
test_fraud['hour'] = test_fraud['trans_datetime'].dt.hour
test_fraud['day_of_week'] = test_fraud['trans_datetime'].dt.dayofweek
test_fraud['day_of_month'] = test_fraud['trans_datetime'].dt.day
test_fraud['is_weekend'] = test_fraud['day_of_week'].isin([5, 6]).astype(int)

# Encoder la catégorie
le_category = LabelEncoder()
test_fraud['category_encoded'] = le_category.fit_transform(test_fraud['category'])

print(" Catégories uniques:")
print(f"Nombre de catégories: {test_fraud['category'].nunique()}")
print("\nTop 10 catégories:")
print(test_fraud['category'].value_counts().head(10))

# Analyser les distances géographiques
print(" Variables géographiques disponibles:")
geo_cols = ['city', 'state', 'street', 'lat', 'long']
for col in geo_cols:
    if col in test_fraud.columns:
        print(f"  - {col}: {test_fraud[col].nunique()} valeurs uniques")

 Colonne de date utilisée: trans_date_trans_time
 Catégories uniques:
Nombre de catégories: 14

Top 10 catégories:
category
gas_transport     56370
grocery_pos       52553
home              52345
shopping_pos      49791
kids_pets         48692
shopping_net      41779
entertainment     40104
personal_care     39327
food_dining       39268
health_fitness    36674
Name: count, dtype: int64
 Variables géographiques disponibles:
  - city: 849 valeurs uniques
  - state: 50 valeurs uniques
  - street: 924 valeurs uniques
  - lat: 910 valeurs uniques
  - long: 910 valeurs uniques


In [23]:
# création des noeuds 
# Cartes
cards = test_fraud['cc_num'].astype(str).unique()
card2id = {c: i for i, c in enumerate(cards)}

# Marchands
merchants = test_fraud['merchant'].unique()
merchant2id = {m: i + len(card2id) for i, m in enumerate(merchants)}
print(merchant2id)
num_nodes = len(card2id) + len(merchant2id)
print("Nombre total de nœuds :", num_nodes)

#créer les arêtes (card <--> marchand)

edge_data = test_fraud.groupby(['cc_num', 'merchant']).agg({
    'amt': ['sum', 'mean', 'count'],
    'is_fraud': 'max'  # is_fraud est utilisé pour labels seulement
}).reset_index()

edges = []
edge_labels = []

for _, row in edge_data.iterrows():
    card_id = card2id[str(row[('cc_num', '')])]
    merchant_id = merchant2id[row[('merchant', '')]]
    edges.append([card_id, merchant_id])
    edge_labels.append(row[('is_fraud', 'max')])

edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

print(f" Arêtes:")
print(f"  Avant agrégation: {len(test_fraud):,} transactions")
print(f"  Après agrégation: {len(edges):,} paires uniques")
print(f"  Réduction: {(1 - len(edges)/len(test_fraud))*100:.1f}%")

# Bidirectionnel
edge_index = torch.cat([edge_index, edge_index.flip(0)], dim=1)
edge_labels = torch.tensor(edge_labels + edge_labels, dtype=torch.float)

print(f"  Après bidirection: {edge_index.shape[1]:,} arêtes")


# construire les features des noeuds 

# features cartes

card_features = test_fraud.groupby('cc_num').agg({
    # Montants
    'amt': ['count', 'sum', 'mean', 'std', 'min', 'max'],
    
    # Temporelles
    'hour': ['mean', 'std'],
    'is_weekend': 'mean',
    'day_of_week': lambda x: x.mode()[0] if len(x.mode()) > 0 else 0,
    
    # Catégories (diversité)
    'category': 'nunique',
    'merchant': 'nunique',
    
    # Géographiques
    'state': 'nunique',
    'city': 'nunique'
}).fillna(0)

# Aplatir les colonnes multi-niveaux
card_features.columns = ['_'.join(col).strip() for col in card_features.columns]

print(" Features des cartes:")
print(card_features.columns.tolist())
print(f"Dimension: {card_features.shape}")
print(" Exemple:")
print(card_features.head())

# features marchands 
merchant_features = test_fraud.groupby('merchant').agg({
    # Montants
    'amt': ['count', 'sum', 'mean', 'std', 'min', 'max'],
    
    # Temporelles
    'hour': ['mean', 'std'],
    'is_weekend': 'mean',
    
    # Diversité clients
    'cc_num': 'nunique',
    
    # Catégorie dominante
    'category': lambda x: le_category.transform([x.mode()[0]])[0] if len(x.mode()) > 0 else 0,
    
    # Géographique
    #'state': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Unknown',
    'city': 'nunique'
}).fillna(0)



merchant_features.columns = ['_'.join(col).strip() if isinstance(col, tuple) else col 
                             for col in merchant_features.columns]

print(" Features des marchands:")
print(merchant_features.columns.tolist())
print(f"Dimension: {merchant_features.shape}")
print(" Exemple:")
print(merchant_features.head())




{'fraud_Kirlin and Sons': 924, 'fraud_Sporer-Keebler': 925, 'fraud_Swaniawski, Nitzsche and Welch': 926, 'fraud_Haley Group': 927, 'fraud_Johnston-Casper': 928, 'fraud_Daugherty LLC': 929, 'fraud_Romaguera Ltd': 930, 'fraud_Reichel LLC': 931, 'fraud_Goyette, Howell and Collier': 932, 'fraud_Kilback Group': 933, 'fraud_Feil, Hilpert and Koss': 934, 'fraud_Gottlieb Group': 935, 'fraud_Connelly-Carter': 936, 'fraud_Bechtelar-Rippin': 937, 'fraud_Lubowitz-Walter': 938, 'fraud_Welch, Rath and Koepp': 939, 'fraud_Hickle Group': 940, 'fraud_Lang, Towne and Schuppe': 941, 'fraud_Morissette LLC': 942, 'fraud_Prosacco LLC': 943, 'fraud_Corwin-Romaguera': 944, 'fraud_Tillman LLC': 945, 'fraud_Veum-Koelpin': 946, 'fraud_Watsica, Haag and Considine': 947, 'fraud_Leannon-Ward': 948, 'fraud_Hintz, Bauch and Smith': 949, 'fraud_Labadie LLC': 950, 'fraud_Eichmann, Hayes and Treutel': 951, 'fraud_Leffler-Goldner': 952, 'fraud_Kautzer and Sons': 953, 'fraud_Ernser-Feest': 954, 'fraud_Zemlak, Tillman and 

In [30]:
# Aligner les dimensions (padding ou sélection)
n_card_features = card_features_final.shape[1]
n_merchant_features = merchant_features_final.shape[1]

print(f" Alignement des dimensions:")
print(f"  Cartes: {n_card_features} features")
print(f"  Marchands: {n_merchant_features} features")

if n_card_features != n_merchant_features:
    print("  Les dimensions diffèrent. Options:")
    print("  1. Padding avec des zéros (simple mais sous-optimal)")
    print("  2. Sélectionner features communes (perte d'info)")
    print("  3. Utiliser un graphe hétérogène (recommandé)")
    
    # Option 1: Padding
    max_features = max(n_card_features, n_merchant_features)
    
    if n_card_features < max_features:
        padding = pd.DataFrame(
            0, 
            index=card_features_final.index,
            columns=[f'pad_{i}' for i in range(max_features - n_card_features)]
        )
        card_features_final = pd.concat([card_features_final, padding], axis=1)
    
    if n_merchant_features < max_features:
        padding = pd.DataFrame(
            0,
            index=merchant_features_final.index,
            columns=[f'pad_{i}' for i in range(max_features - n_merchant_features)]
        )
        merchant_features_final = pd.concat([merchant_features_final, padding], axis=1)


 Alignement des dimensions:
  Cartes: 18 features
  Marchands: 18 features


In [31]:
# création de l'objet graphe qui est l'entrée dans le vgae
data_test = Data(
    x=X_tensor,
    edge_index=edge_index,
    y=edge_labels,  # Labels pour validation
    num_nodes=num_nodes
)

print("\n" + "="*80)
print("GRAPHE FINAL")
print("="*80)
print(data_test)

print(f"""
RÉSUMÉ:
  • Nœuds: {num_nodes:,}
  • Features par nœud: {X_tensor.shape[1]}
  • Arêtes: {edge_index.shape[1]:,}
  • Labels disponibles: {len(edge_labels):,}
  • Pas de data leakage: ✓
  • Features normalisées: ✓
  • Multi-edges agrégées: ✓
""")


GRAPHE FINAL
Data(x=[1676, 18], edge_index=[2, 656792], y=[656792], num_nodes=1617)

RÉSUMÉ:
  • Nœuds: 1,617
  • Features par nœud: 18
  • Arêtes: 656,792
  • Labels disponibles: 656,792
  • Pas de data leakage: ✓
  • Features normalisées: ✓
  • Multi-edges agrégées: ✓



In [32]:
model.eval()
data_test = data_test.to(device)

with torch.no_grad():
    logits_test = model(data_test)

preds_test = logits_test.argmax(dim=1)

# Si labels disponibles
if hasattr(data_test, "y"):
    data_test.y = data_test.y.long()
    acc_test = (preds_test == data_test.y).float().mean().item()
    print(f"Test Accuracy: {acc_test:.4f}")

# Probabilités de fraude
probas_test = torch.softmax(logits_test, dim=1)[:, 1]


TypeError: GCNEncoder.forward() missing 1 required positional argument: 'edge_index'

# Application du VGAE 

In [15]:

# création de la matrice adjacente
adj = sp.coo_matrix(
    (np.ones(edge_index.shape[1]),
     (edge_index[0].numpy(), edge_index[1].numpy())),
    shape=(num_nodes, num_nodes)
)


In [16]:
def sparse_to_tuple(sparse_mx):
    """
    Convertit une matrice sparse scipy en tuple (coords, values, shape)
    """
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()

# indice des arêtes
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()

#poids normalisés
    values = sparse_mx.data

#taille de la matrice 
    shape = sparse_mx.shape

    return coords, values, shape


def preprocess_graph(adj):
    """
    Prétraitement du graphe pour VGAE / GCN
    - Ajoute des self-loops
    - Normalise la matrice d'adjacence
    - Retourne une représentation sparse exploitable
    """

    # S'assure que la matrice est en format COO
    adj = sp.coo_matrix(adj)

    # Ajout des self-loops
    adj_ = adj + sp.eye(adj.shape[0])

    # Calcul du degré de chaque nœud
    rowsum = np.array(adj_.sum(1))

    # D^{-1/2}
    degree_mat_inv_sqrt = sp.diags(
        np.power(rowsum, -0.5).flatten()
    )

    # Normalisation symétrique : D^{-1/2} (A + I) D^{-1/2}
    adj_normalized = adj_.dot(degree_mat_inv_sqrt)\
                         .transpose()\
                         .dot(degree_mat_inv_sqrt)\
                         .tocoo()

    return sparse_to_tuple(adj_normalized)


In [17]:
class GCNEncoder(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, 2 * out_channels)
        self.conv_mu = GCNConv(2 * out_channels, out_channels)
        self.conv_logstd = GCNConv(2 * out_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index).relu()
        mu = self.conv_mu(x, edge_index)
        logstd = self.conv_logstd(x, edge_index)
        return mu, logstd

# InnerProductDecoder suppose un graphe non dirigé 

# modèle
out_channels = 32
encoder = GCNEncoder(data_train.num_features, out_channels)
model = VGAE(encoder, decoder=InnerProductDecoder())

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

for epoch in range(200):
    model.train()
    optimizer.zero_grad()

    z = model.encode(data_train.x, data_train.edge_index)

    loss = model.recon_loss(z, data_train.edge_index)
    loss = loss + model.kl_loss()

    loss.backward()
    optimizer.step()

    print(f"Epoch {epoch}, Loss: {loss.item():.4f}")



Epoch 0, Loss: 12.6578
Epoch 1, Loss: 7.1636
Epoch 2, Loss: 5.2483
Epoch 3, Loss: 4.8220
Epoch 4, Loss: 4.7015
Epoch 5, Loss: 4.6714
Epoch 6, Loss: 4.6848
Epoch 7, Loss: 4.5419
Epoch 8, Loss: 4.5271
Epoch 9, Loss: 4.4833
Epoch 10, Loss: 4.4629
Epoch 11, Loss: 4.4086
Epoch 12, Loss: 4.4008
Epoch 13, Loss: 4.3396
Epoch 14, Loss: 4.3973
Epoch 15, Loss: 4.3431
Epoch 16, Loss: 4.3117
Epoch 17, Loss: 4.2520
Epoch 18, Loss: 4.3142
Epoch 19, Loss: 4.3112
Epoch 20, Loss: 4.2614
Epoch 21, Loss: 4.2914
Epoch 22, Loss: 4.2723
Epoch 23, Loss: 4.2963
Epoch 24, Loss: 4.3032
Epoch 25, Loss: 4.2735
Epoch 26, Loss: 4.2858
Epoch 27, Loss: 4.2383
Epoch 28, Loss: 4.2708
Epoch 29, Loss: 4.2401
Epoch 30, Loss: 4.2678
Epoch 31, Loss: 4.2486
Epoch 32, Loss: 4.2282
Epoch 33, Loss: 4.2693
Epoch 34, Loss: 4.2442
Epoch 35, Loss: 4.2209
Epoch 36, Loss: 4.2272
Epoch 37, Loss: 4.2817
Epoch 38, Loss: 4.2489
Epoch 39, Loss: 4.2179
Epoch 40, Loss: 4.2372
Epoch 41, Loss: 4.2469
Epoch 42, Loss: 4.2201
Epoch 43, Loss: 4.23

In [18]:
# score d'anomalie 
# récupérer les embeddings
 
model.eval()

with torch.no_grad():
    # embeddings latents des noeuds
    z = model.encode(data_train.x, data_train.edge_index)

    # score de reconstruction pour chaque arête EXISTANTE
    recon_scores = model.decoder(
        z,
        data_train.edge_index
    ).sigmoid().view(-1)


In [19]:
#calcul de l'AUC fraude

y_true = edge_labels.view(-1).cpu().numpy()
y_score = recon_scores.cpu().numpy()

auc = roc_auc_score(y_true, y_score)
print("AUC fraude :", auc)

AUC fraude : 0.5459828251206486


Le VGAE ne capte pas bien la fraude pour ce cas de graphe 

## Graphe transaction <--> transaction

Pour transformer des lignes de CSV en un réseau, nous devons définir ce qui lie deux transactions entre elles. Généralement, on crée une arête si deux transactions partagent une propriété commune dans un intervalle de temps restreint.

In [20]:

# 1. Chargement et Préparation
df = pd.read_csv("../data/SimulatedfraudTrain.csv")

# Encodage minimal pour les features des nœuds (transactions)
le_cat = LabelEncoder()
df['cat_enc'] = le_cat.fit_transform(df['category'])

# Sélection des features numériques pour chaque nœud
features = ['amt', 'lat', 'long', 'city_pop', 'cat_enc']
x_raw = df[features].values

# Normalisation (Indispensable comme vu précédemment)
scaler = StandardScaler()
x_norm = scaler.fit_transform(x_raw)
x = torch.tensor(x_norm, dtype=torch.float)

# 2. Création des Arêtes (Transactions liées par la même carte)
# On trie par date pour lier les transactions chronologiquement
df = df.sort_values(['cc_num', 'trans_date_trans_time'])
df['index_original'] = range(len(df))

# On crée des liens entre la transaction T et T+1 pour une même carte
edge_sources = []
edge_targets = []

# Groupement par carte pour créer les liens séquentiels
for cc, group in df.groupby('cc_num'):
    indices = group['index_original'].values
    if len(indices) > 1:
        # Lien de T_i vers T_{i+1}
        edge_sources.extend(indices[:-1])
        edge_targets.extend(indices[1:])

edge_index = torch.tensor([edge_sources, edge_targets], dtype=torch.long)

# 3. Création des Labels (is_fraud)
y = torch.tensor(df.sort_values('index_original')['is_fraud'].values, dtype=torch.long)

# 4. Construction de l'objet PyG
data = Data(x=x, edge_index=edge_index, y=y)

print(f"Nombre de nœuds (transactions) : {data.num_nodes}")
print(f"Nombre d'arêtes (liens temporels) : {data.num_edges}")

Nombre de nœuds (transactions) : 1296675
Nombre d'arêtes (liens temporels) : 1295692
